In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
is_out_exist = os.path.exists('out')
if not is_out_exist:
    os.makedirs('out')
    print("The new directory is created!")
reference_image_name = '.\\img\\40.jpg'
target_image_name    = '.\\img\\42.jpg'

In [ ]:
def generate_search_window(i,j,p, shape):
    lu = (max(0, i-p-1), max(0, j-p-1))               #i-p, j-p
    ld = (min(shape[0], i+p), max(0, j-p-1))        #i+p, j-p
    ru = (max(0, i-p-1), min(shape[1], j+p))        #i-p, j+p
    rd = (min(shape[0], i+p), min(shape[1], j+p)) #i+p, j+p
    return lu, ld, ru, rd
def SAD(window, blk):
    return np.sum(np.abs(window-blk))
def full_search          (reference_image, target_image,u,v,p, N):
    min_SAD = 100000000000000
    u_goto = 0
    v_goto = 0
    max_row = reference_image.shape[0]
    max_col = reference_image.shape[1]
    for i in range(-p, p+1):
        for j in range(-p, p+1):
            ref_lu_corner = (i+u,j+v)
            if ref_lu_corner[0] < 0 or ref_lu_corner[1] < 0 or ref_lu_corner[0]+N > max_row or ref_lu_corner[1]+N > max_col:
                continue
            tmp = SAD(target_image[u:u+N,v:v+N,:], reference_image[ref_lu_corner[0]:ref_lu_corner[0]+N, ref_lu_corner[1]:ref_lu_corner[1]+N, :])
            if tmp < min_SAD:
                min_SAD = tmp
                u_goto = i
                v_goto = j
    return u_goto, v_goto
def logarithmic_search_2d(reference_image, target_image,u,v,p, N):
    #print('at', p, 'u,v=',u,v)
    min_SAD = 100000000000000
    u_goto = 0
    v_goto = 0
    max_row = reference_image.shape[0]
    max_col = reference_image.shape[1]
    for i in [-p, 0, p+1]:
        for j in [-p, 0, p+1]:
            ref_lu_corner = (i+u,j+v)
            if ref_lu_corner[0] < 0 or ref_lu_corner[1] < 0 or ref_lu_corner[0]+N > max_row or ref_lu_corner[1]+N > max_col:
                continue
            tmp = SAD(target_image[u:u+N,v:v+N,:], reference_image[ref_lu_corner[0]:ref_lu_corner[0]+N, ref_lu_corner[1]:ref_lu_corner[1]+N, :])
            if tmp < min_SAD:
                min_SAD = tmp
                u_goto = i
                v_goto = j
    if p == 1:
        return u+u_goto, v+v_goto
    else:
        return logarithmic_search_2d(reference_image, target_image,u+u_goto,v+v_goto,int(p/2), N)
def my_motion_estimation(reference_image_name, target_image_name, N, p, approach):
    reference_image = cv2.imread(reference_image_name, cv2.IMREAD_COLOR)
    target_image = cv2.imread(target_image_name, cv2.IMREAD_COLOR)
    motion_vectors = np.zeros((int(reference_image.shape[0]/N), int(reference_image.shape[1]/N), 2))
    residual_image = np.zeros((reference_image.shape[0], reference_image.shape[1], 3))
    predicted_image = np.zeros((reference_image.shape[0], reference_image.shape[1], 3))
    max_row = reference_image.shape[0]
    max_col = reference_image.shape[1]
    for u in range(0,max_row, N):
        for v in range(0, max_col, N):
            if approach == 'full':
                u_goto, v_goto = full_search          (reference_image, target_image,u,v,p,N)
                motion_vectors[int(u/N), int(v/N)] = (u_goto, v_goto)
                #print(u+u_goto, u+u_goto+N, v+v_goto, v+v_goto+N)
                #print(reference_image[u+u_goto:u+u_goto+N, v+v_goto:v+v_goto+N, : ].shape, reference_image.shape)            
                predicted_image[u:u+N, v:v+N,:] = reference_image[u+u_goto:u+u_goto+N, v+v_goto:v+v_goto+N, : ].copy()
            else:
                u_goto, v_goto = logarithmic_search_2d(reference_image, target_image,u,v,p,N)
                motion_vectors[int(u/N), int(v/N)] = (u_goto-u, v_goto-v)
                predicted_image[u:u+N, v:v+N,:] = reference_image[u_goto:u_goto+N, v_goto:v_goto+N, : ].copy()
    #print(motion_vectors)
    residual_image = target_image - predicted_image
    return motion_vectors, residual_image, predicted_image
def draw_arrow_line(estimated_motion, reference_image_name, N, vector_image_name):
    img = cv2.imread(reference_image_name, cv2.IMREAD_COLOR)
    for i in range(0,img.shape[0], N):
        for j in range(0,img.shape[1], N):
            start = (i,j)
            end = (int(i+estimated_motion[int(i/N), int(j/N)][0]),int(j + estimated_motion[int(i/N), int(j/N)][1]))
            
            start = (start[1], start[0])
            end = (end[1], end[0])
            cv2.arrowedLine(img, end, start, (0,0,225))
    cv2.imwrite(vector_image_name, img)
    #cv2.imshow('arrow line image', img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
def draw_residual(residual_image, residul_image_name):
    gray_residual = np.zeros((residual_image.shape[0], residual_image.shape[1],1))
    for i in range(residual_image.shape[0]):
        for j in range(residual_image.shape[1]):
            gray_residual[i,j] = 0.2989*residual_image[i,j,0] + 0.5870*residual_image[i,j,1] + 0.1140*residual_image[i,j,2]
    cv2.imwrite(residul_image_name, gray_residual)
    #cv2.imshow('residual image', gray_residual)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
def draw_predicted_image(predicted_image, predicted_image_name):
    cv2.imwrite(predicted_image_name, predicted_image)
    #cv2.imshow('predicted image', predicted_image)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

In [ ]:
for approach in ['full','2d']:
    for N in [8,16]:
        for p in [8,16]:
            estimated_motion, residual_image, predicted_image = my_motion_estimation(reference_image_name=reference_image_name, target_image_name=target_image_name, N=N, p=p, approach=approach)
            vector_image_name = 'out\\'+approach+'_motion_vection_r'+str(N)+'_p'+str(p)+'.png'
            residual_image_name = 'out\\'+approach+'_residual_r'+str(N)+'_p'+str(p)+'.png'
            predicted_image_name = 'out\\'+approach+'_predicted_r'+str(N)+'_p'+str(p)+'.png'
            draw_residual(residual_image, residul_image_name=residual_image_name)
            draw_arrow_line(estimated_motion, target_image_name, N=N, vector_image_name=vector_image_name)
            draw_predicted_image(predicted_image, predicted_image_name=predicted_image_name)